In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
color = sns.color_palette()
%matplotlib inline
matplotlib.style.use('ggplot')

import time
import numpy as np
import pandas as pd
from IPython.display import display

# remove warnings
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve
from itertools import product

from models.get_datasets import load_stacking_datasets

In [2]:
train, test = load_stacking_datasets()

load stacking features


In [3]:
X_train = train.drop(['Id','Score'], axis = 1)
X_test  = test.drop(['Id'], axis = 1)

y_train = train['Score']

df_columns = X_train.columns
print('特征数 %d' % len(df_columns))

特征数 350


In [4]:
dtrain = xgb.DMatrix(X_train.values, y_train.values, feature_names=df_columns)
dtest = xgb.DMatrix(X_test.values, feature_names=df_columns)

In [18]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve
from itertools import product

def model_cross_validate(xgb_params, cv_param_dict, dtrain, cv_num_boost_round=4000, early_stopping_rounds=100, cv_nfold=5, stratified=True):
    params_value = []
    params_name = [k for k, w in cv_param_dict.items()]
    max_auc = 0
    for param in params_name:
        params_value.append(cv_param_dict[param])

    for param_pair in product(*params_value):
        param_str = ''
        for i in range(len(param_pair)):
            param_str += params_name[i] + '=' + str(param_pair[i]) + ' '
            xgb_params[params_name[i]] = param_pair[i]
        
        start = time.time()
        cv_result = xgb.cv(xgb_params, dtrain, num_boost_round=cv_num_boost_round, stratified=stratified,
                           nfold=cv_nfold, early_stopping_rounds=early_stopping_rounds)
        
        best_num_boost_rounds = len(cv_result)
        mean_test_auc = cv_result.loc[best_num_boost_rounds - 6: best_num_boost_rounds - 1, 'test-rmse-mean'].mean()
        mean_test_auc = 1 / (1 + mean_test_auc)
        
        if mean_test_auc > max_auc:
            best_param = param_pair
            max_auc = mean_test_auc
        
        end = time.time()
        print('Tuning paramter: {}, best_ntree_limit:{}, auc = {:.7f}, cost: {}s'.format(param_str, best_num_boost_rounds,
                                                                              mean_test_auc, end-start))
    param_str = ''
    for i in range(len(best_param)):
        param_str += params_name[i] + '=' + str(best_param[i]) + ' '
        xgb_params[params_name[i]] = best_param[i]
    print('===========best paramter: {} auc={:.7f}==========='.format(param_str, max_auc))

In [6]:
xgb_params = {
    'eta': 0.1,
    'min_child_weight': 1,
    'colsample_bytree': 0.8,
    'max_depth': 5,
    'subsample': 0.8,
    'gamma': 0,
    'scale_pos_weight': 1,
    'eval_metric': 'rmse',
    'objective': 'reg:linear',
    'seed':2018,
    'silent': 1,
    'booster': 'gbtree'
}

In [10]:
print('---> calc baseline model')

cv_num_boost_round=5000
early_stopping_rounds=100
cv_nfold=5
stratified=True

cv_result = xgb.cv(xgb_params,
                   dtrain,
                   nfold=cv_nfold,
                   stratified=stratified,
                   num_boost_round=cv_num_boost_round,
                   early_stopping_rounds=early_stopping_rounds,
                   )

best_num_boost_rounds = len(cv_result)
mean_train_auc = cv_result.loc[best_num_boost_rounds-6 : best_num_boost_rounds-1, 'train-rmse-mean'].mean()
mean_test_auc = cv_result.loc[best_num_boost_rounds-6 : best_num_boost_rounds-1, 'test-rmse-mean'].mean()

mean_train_auc = 1 / (1 + mean_train_auc)
mean_test_auc = 1 / (1 + mean_test_auc)

print('mean_train_auc = {:.7f} , mean_test_rmse = {:.7f}\n'.format(mean_train_auc, mean_test_auc))

---> calc baseline model
mean_train_auc = 0.6361679 , mean_test_rmse = 0.6220402



In [19]:
cv_paramters = {'max_depth':range(5,15,2),'min_child_weight':range(1,10,2)}
model_cross_validate(xgb_params, cv_paramters, dtrain)

Tuning paramter: max_depth=5 min_child_weight=1 , best_ntree_limit:120, auc = 0.6220402, cost: 336.6737847328186s
Tuning paramter: max_depth=5 min_child_weight=3 , best_ntree_limit:173, auc = 0.6220597, cost: 409.0248680114746s
Tuning paramter: max_depth=5 min_child_weight=5 , best_ntree_limit:73, auc = 0.6220327, cost: 314.0649356842041s


KeyboardInterrupt: 